In [1]:
import scripts.init_gpu as init_gpu
import scripts.init_dataset as init_dataset
import pandas as pd
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
init_gpu.initialize_gpus()

locations = ['LOC1', 'LOC2']

print("Loading Dataset...")
# load the dataset
df = pd.read_csv(
    f"../dataset/processed/{locations[0]}-{locations[1]}-scaled-balanced.csv")

length = len(df.columns) - 2  # subtract the two label columns

# get train-test set
train_df, test_df, train_web_sam1ples, test_web_samples = init_dataset.get_sample(
    df, locations, range(1500), 1200)

train_df.head()

2024-11-21 11:00:15.840049: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 11:00:15.847690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732147215.856201 4087534 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732147215.858775 4087534 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 11:00:15.867797: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Num GPUs Available:  1
Loading Dataset...
Training Websites: [1309, 228, 51, 563, 501, 457, 285, 209, 1385, 1116, 178, 1209, 864, 65, 61, 191, 447, 476, 1034, 1232, 54, 1149, 407, 1466, 1330, 1436, 1490, 859, 451, 919, 1206, 569, 13, 326, 1429, 865, 696, 1468, 318, 440, 689, 1492, 189, 778, 198, 735, 704, 1236, 541, 88, 940, 1098, 255, 775, 161, 1130, 600, 1287, 1266, 740, 1182, 393, 142, 93, 1354, 466, 592, 163, 1482, 206, 1456, 1462, 928, 1301, 747, 333, 758, 727, 429, 1372, 546, 1399, 1327, 146, 1247, 1300, 350, 1093, 1495, 334, 946, 777, 552, 1310, 1140, 449, 1402, 664, 114, 469, 1486, 646, 821, 548, 135, 432, 1161, 644, 435, 1342, 1022, 810, 1316, 939, 292, 542, 1493, 505, 1478, 1103, 538, 1197, 877, 1195, 817, 741, 1404, 283, 1043, 1010, 186, 96, 224, 313, 1285, 327, 1487, 1221, 130, 788, 781, 1220, 958, 1083, 514, 1133, 23, 234, 1099, 1419, 1312, 1463, 1498, 601, 890, 323, 929, 6, 539, 1025, 365, 1039, 217, 1280, 611, 1308, 1338, 1415, 1477, 1366, 765, 330, 1104, 1086, 1, 1226, 

/home/asil0892/doh_traffic_analysis/code/scripts/init_dataset.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.sort_values(by=["Location"], inplace=True)


,Location,Website,0,1,2,3,4,5,6,7,...,116,117,118,119,120,121,122,123,124,125
0,LOC1,0,-1.086861,0.690199,0.647933,0.201517,0.207139,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604
1,LOC1,1005,1.095547,-1.585943,0.647933,0.343653,0.207139,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604
2,LOC1,1005,1.095547,0.780047,-1.972790,0.343653,0.207139,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604
3,LOC1,1005,-1.086861,0.690199,0.647933,0.343653,0.207139,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604
4,LOC1,1005,1.095547,0.780047,0.183501,-0.828965,-2.083179,-0.031855,-0.316768,0.824649,...,0.055571,0.160203,0.121677,0.093533,0.09274,0.057062,0.155991,0.109839,0.086282,0.08604


Preprocessing

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import random
random.seed(42)
selected_location = 'LOC1'
selected_websites = random.sample(test_web_samples, 100)
location_df = test_df[(test_df['Location'] == selected_location) & (test_df['Website'].isin(selected_websites))]
X = location_df.iloc[:, 2:].to_numpy()

le = LabelEncoder()
y = le.fit_transform(location_df.Website)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from scripts.classification import evaluate_classification_model

rf = RandomForestClassifier()
evaluate_classification_model(X_train, y_train, X_test, y_test, rf)

Accuracy: 92.05, F1 Score:  93.16, Precision:  94.96, Recall:  92.03


(0.9204545454545454,
 0.9495556758091045,
 0.9202539814873415,
 0.9315588838108624,
 array([[60,  0,  0, ...,  0,  0,  0],
        [ 0, 60,  0, ...,  0,  0,  0],
        [ 0,  0, 60, ...,  0,  0,  1],
        ...,
        [ 0,  0,  0, ..., 51,  0,  0],
        [ 0,  0,  0, ...,  0, 56,  0],
        [ 0,  0,  0, ...,  0,  0, 61]]))

# check accuracy after reconstruction of vae model

In [219]:
from scripts.train_vae import VAE, Sampling, ConvVAE_BatchNorm
import tensorflow as tf

vae_model = tf.keras.models.load_model("../models/vae/ci_vae/ConvBased/domain_and_class/LOC1-LOC2-e880-mse1-kl0.01-cl1.0-ConvBatchNorm-ldim96-hdim128.keras", custom_objects={'ConvVAE_BatchNorm': ConvVAE_BatchNorm, 'Sampling': Sampling})


/home/asil0892/doh_traffic_analysis/.venv/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [224]:
X_test_recon, _, _ = vae_model(X_test)
rf.score(X_test_recon, y_test)

0.7315151515151516

In [196]:
X_train_recon, _, _ = vae_model(X_train)
rf.score(X_train_recon, y_train)

0.7956716417910448

Retrain model with reconstructed data and actual data and see if the original accuracy retains

In [20]:
import numpy as np
X_train_recon, _, _ = vae_model(X_train)


X_train_combined = np.vstack((X_train_recon, X_train))
y_train_combined = np.hstack((y_train, y_train))

# test on actual data
evaluate_classification_model(X_train_combined, y_train_combined, X_test, y_test, rf)

Accuracy: 90.30, F1 Score:  92.20, Precision:  95.07, Recall:  90.31


(0.9030303030303031,
 0.9506717684732965,
 0.9030767716687588,
 0.9220381447670826,
 array([[58,  0,  0, ...,  0,  0,  0],
        [ 0, 63,  0, ...,  0,  0,  0],
        [ 0,  0, 58, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ..., 54,  0,  0],
        [ 0,  0,  0, ...,  0, 58,  0],
        [ 0,  0,  0, ...,  0,  0, 36]]))

In [21]:
rf.score(X_test_recon, y_test)

0.7522727272727273